In [1]:
from ultralytics import YOLO
from ultralytics.utils.metrics import mask_iou
from torch import Tensor
import cv2
import numpy as np
import pandas as pd

In [2]:
model = YOLO('./runs/segment/train2/weights/best.pt')
img_size = 1024

In [3]:
def read_and_normalize_mask(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)

    _, mask = cv2.threshold(image, 0.5, 1, cv2.THRESH_BINARY)

    mask = mask.reshape(-1, int(image.shape[1] * image.shape[0]))

    return Tensor(mask)

def get_iou(yolo_model, img_path):
    results = yolo_model(img_path)
    true_mask = read_and_normalize_mask(img_path.replace("images", "masks"))
    
    assert len(results) == 1, "support only len == 1"
    for r in results:
        if isinstance(r.masks, type(None)): return None
        masks = r.masks.data.reshape(-1, img_size*img_size).cpu()
        masks = np.any(np.array(masks), axis=0)
        masks = masks.reshape(-1, img_size*img_size)
        print(masks)
        masks = Tensor(masks)
    
        return mask_iou(true_mask, masks)


In [4]:
# get an average iou
import os
import numpy as np

image_directory = "/dev/bakalarka-yolo/data/ch-test(SOHO)/images/"

iou_values = []
for image_path in os.listdir(image_directory):
    if image_path.endswith(".png"):
        result = get_iou(model, os.path.join(image_directory, image_path))
        if result != None:
            iou = result.tolist()[0][0]
            iou_values.append(iou)

average_iou = np.mean(iou_values)
median = np.median(iou_values)


image 1/1 /dev/bakalarka-yolo/data/ch-test(SOHO)/images/20171114_1313_eit195_1024.png: 1024x1024 3 coronal holes, 72.8ms
Speed: 3.6ms preprocess, 72.8ms inference, 2.8ms postprocess per image at shape (1, 3, 1024, 1024)
[[False False False ... False False False]]

image 1/1 /dev/bakalarka-yolo/data/ch-test(SOHO)/images/20110105_0113_eit195_1024.png: 1024x1024 5 coronal holes, 72.8ms
Speed: 2.0ms preprocess, 72.8ms inference, 1.5ms postprocess per image at shape (1, 3, 1024, 1024)
[[False False False ... False False False]]

image 1/1 /dev/bakalarka-yolo/data/ch-test(SOHO)/images/20161026_1313_eit195_1024.png: 1024x1024 4 coronal holes, 72.8ms
Speed: 2.0ms preprocess, 72.8ms inference, 1.5ms postprocess per image at shape (1, 3, 1024, 1024)
[[False False False ... False False False]]

image 1/1 /dev/bakalarka-yolo/data/ch-test(SOHO)/images/20130427_0113_eit195_1024.png: 1024x1024 3 coronal holes, 69.6ms
Speed: 2.1ms preprocess, 69.6ms inference, 1.5ms postprocess per image at shape (1,

In [5]:
%%capture cell
val_res = model.val(data='./ch-SOHO.yaml', imgsz=1024, split="test")

Ultralytics YOLOv8.1.23 🚀 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA TITAN RTX, 24210MiB)
val: New cache created: /dev/bakalarka-yolo/data/ch-test(SOHO)/images-col.cache
                   all       1977       2930      0.705      0.498       0.58      0.341      0.732      0.466      0.546      0.253
Speed: 0.5ms preprocess, 59.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/segment/val2


In [6]:
dice = val_res.seg.f1
print("Dice score:", dice[0])
print("IoU:", average_iou)

#  results for ch colorful images
# Dice score: 0.5807742979901169
# IoU: 0.7006662135058757

# results for ch grayscale
# Dice score: 0.5563138232924477
# IoU: 0.6986303846845785

# results for ar colorful
# Dice score: 0.2744738541877972
# IoU: 0.3191702537726108

# results for ar grayscale
# Dice score: 0.2924788789771119
# IoU: 0.32751943564349206

# results for ch soho grayscale
# Dice score: 0.569427672482262
# IoU: 0.6698140347881835

# results for ar soho grayscale
# Dice score: 0.44100682104197647
# IoU: 0.46968485340150384

Dice score: 0.569427672482262
IoU: 0.6698140347881835


In [24]:






!cp /dev/bakalarka-yolo/data/ar-test\(SOHO\) data/ar-test\(SOHO\) -r